In [1]:
import requests
import csv
import cx_Oracle

# function to fetch data from a public API data source
def fetch_data_from_api(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to fetch data from the API.")
        return None

# function to flatten json data into csv format
def flatten_json_to_csv(data, csv_file):
    if data:
        with open(csv_file, 'w', newline='') as csvfile:
            # create a csv dict object with field names from the json keys
            writer = csv.DictWriter(csvfile, fieldnames=data[0].keys())
            writer.writeheader()
            writer.writerows(data)
            print("JSON data flattened and saved to CSV successfully.")
    else:
        print("No data to flatten.")

# function to load csv data into a relational database
def load_csv_to_rdb(csv_file, rdb_connection_string, table_name):
    try:
        connection = cx_Oracle.connect(rdb_connection_string)
        cursor = connection.cursor()
        with open(csv_file, 'r') as csv_file:
            next(csv_file)
            csv_reader = csv.reader(csv_file)
            cursor.executemany(f"INSERT INTO {table_name} VALUES ({','.join([':{}'.format(i+1) for i in range(len(next(csv_reader)))])})", csv_reader)
        connection.commit()
        print("CSV data loaded into table successfully.")
    except Exception as e:
        print(f"Error loading data into table: {e}")
    finally:
        cursor.close()
        connection.close()

# main to execute the three functions sequentially
def main():
    api_url = "https://softwium.com/api/books"
    csv_file = "books.csv"
    rdb_connection_string = "nbc/xx12@localhost:1521/nbc_eval"
    table_name = "python_schema.books_table"

    data = fetch_data_from_api(api_url)
    flatten_json_to_csv(data, csv_file)
    load_csv_to_rdb(csv_file, rdb_connection_string, table_name)

# execution
if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'cx_Oracle'